In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import numpy as np
import torch
import scanpy as sc
import muon as mu
import anndata
import mudata
import pandas as pd

In [8]:
%load_ext autoreload
%autoreload 2

In [62]:
mdata = mu.read("data/mdata_preprocessed_final_5k_obs.h5mu")
mdata

MuData object with n_obs × n_vars = 5000 × 41910
  obs:	'imagerow', 'imagecol', 'ann', 'sample', 'patient'
  2 modalities
    rna:	5000 x 29400
      obs:	'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'size_factors', 'sample', 'type', 'patient', 'clusters', 'tissue', 'imagerow', 'imagecol', 'ann'
      obsm:	'X_pca', 'X_umap', 'spatial'
      layers:	'counts', 'scran_normalization', 'spotclean'
    msi:	5000 x 12510
      obs:	'ST_barcode', 'sample', 'tissue', 'imagerow', 'imagecol', 'ann', 'patient'
      var:	'mz_values'
      obsm:	'spatial'

In [63]:
from src.model import MVAE, MVAEParams

mvae_params = MVAEParams(beta=0.01, n_layers=2, z_dim=100, n_hidden=300)

In [64]:
torch.set_num_threads(16)
model = MVAE(mdata, mvae_params, use_cuda=True)
model

N batches for mod1:  16
N batches for mod2:  16
(5000, 29400)
(5000, 12510)


MVAE(
  (rna): ModalityLayers(
    (shared_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (batch_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (private_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.

In [66]:
from src.train import train_mvae, TrainParams
train_params = TrainParams(train_size=0.8, n_epochs=100, learning_rate=1e-4, batch_size=128)

In [73]:
corr = train_mvae(model, mdata, train_params)

Train data size: 4000
Test data size: 1000


  0%|                                                    | 0/32 [00:00<?, ?it/s]/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
  0%|                                                    | 0/32 [00:02<?, ?it/s]


KeyError: 'sample'

In [28]:
msi = anndata.AnnData(data["msi"].squeeze().numpy())
msi

AnnData object with n_obs × n_vars = 128 × 12510

In [30]:
a = mudata.MuData({"rna": rna, "msi": msi})
a

/media/gambino/students_workdir/anna/miniconda3/envs/ak/lib/python3.9/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


MuData object with n_obs × n_vars = 128 × 41910
  2 modalities
    rna:	128 x 29400
    msi:	128 x 12510

In [31]:
sc.pp.neighbors(a)
sc.tl.umap(a)

         Falling back to preprocessing with `sc.pp.pca` and default params.


AttributeError: 'NoneType' object has no attribute 'dtype'